## 랭체인 Multi-Chain 구현
- LCEL(LangChain Expression Language)로 체인 만들기

In [1]:
from dotenv import load_dotenv
import os

# .env 파일 불러오기
load_dotenv("C:/env/.env")

True

#### 순차 연결 (Pipeline with | Operator)

In [8]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate

# LLM 모델 선택
llm = ChatOpenAI(model="gpt-4o-mini")

# 1단계: 텍스트 요약 프롬프트
summary_prompt = PromptTemplate.from_template(
    "다음 텍스트를 한 문장으로 요약해줘:\n{text}"
)

# 2단계: 영어 번역 프롬프트
translate_prompt = PromptTemplate.from_template(
    "다음 문장을 영어로 번역:\n{summary}"
)

# 체인 구성 : 요약 -> 번역
multi_chain = summary_prompt | llm | translate_prompt | llm

# 실행
result = multi_chain.invoke({"text":"랭체인은 LLM 애플리케이션을 쉽게 만들 수 있는 프레임워크이다."})
print(result.content)

The translation of the given sentence is: "LangChain is a framework that helps to easily develop LLM applications."


In [5]:
print(summary_prompt)
print(translate_prompt)

input_variables=['text'] input_types={} partial_variables={} template='다음 텍스트를 한 문장으로 요약해줘:\n{text}'
input_variables=['summary'] input_types={} partial_variables={} template='다음 문장을 영어로 번역:\n{summary}'


#### 조건 분기 (Router / Multi-Prompt Chain)

In [ ]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnableBranch
from langchain_core.output_parsers import StrOutputParser

llm = ChatOpenAI(model="gpt-4o-mini")
parser = StrOutputParser()

# 체인 정의
